In [1]:
import sys
sys.path.append('..')

In [2]:
import tqdm
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy

from src.data import *
from src.model import *
from src.recourse import *
from src.utils import *

warnings.filterwarnings('ignore')

In [3]:
def append_result(d, alg_name, seed, alpha, lamb, i, x_0, theta_0, beta, x_r, theta_r, p, theta_p, J_r, J_c, robustness, consistency):
    d['alg'].append(alg_name)
    d['seed'].append(seed)
    d['alpha'].append(alpha)
    d['lambda'].append(lamb)
    d['i'].append(i)
    d['x_0'].append(x_0.round(4))
    d['theta_0'].append(theta_0.round(4))
    d['beta'].append(beta)
    d['x_r'].append(x_r.round(4))
    d['theta_r'].append(theta_r.round(4))
    d['p'].append(p)
    d['theta_p'].append(theta_p.round(4))
    d['J_r'].append(J_r)
    d['J_c'].append(J_c)
    d['robustness'].append(robustness)
    d['consistency'].append(consistency)

In [4]:
def recourse_runner(seed: int, X: np.ndarray, lar_recourse: LARRecourse, roar_recourse: ROAR, params: dict, dataset: Dataset, predictions: List):
    alpha = params['alpha']
    lamb = params['lamb']
    params['algs'] = [alg.lower() for alg in params['algs']]
    betas = np.arange(0., 1.01, 0.01).round(2)
    
    results_opt = {'alg': [], 'seed': [], 'alpha': [], 'lambda': [], 'i': [], 'x_0': [], 'theta_0': [], 'beta': [], 'x_r': [], 'theta_r': [], 'p': [], 'theta_p': [], 'J_r': [], 'J_c': [], 'robustness': [], 'consistency': []}
    results_roar = deepcopy(results_opt)
    weights_0, bias_0 = lar_recourse.weights, lar_recourse.bias
    theta_0 = np.hstack((weights_0, bias_0))
    
    n = len(X)
    for i in tqdm.trange(n, desc=f'Evaluating recourse | alpha={alpha}; lambda={lamb}', colour='#0091ff'):
        x_0 = X[i]
        J = RecourseCost(x_0, lamb)
        
        # Robust Recourse
        x_r = lar_recourse.get_recourse(x_0, beta=1.)
        weights_r, bias_r = lar_recourse.calc_theta_adv(x_r)
        theta_r = np.hstack((weights_r, bias_r))
        J_r_opt = J.eval(x_r, weights_r, bias_r)
        
        for p, prediction in enumerate(predictions):
            weights_p, bias_p = prediction[:-1], prediction[[-1]]
            theta_p = (weights_p, bias_p)
            
            # Consistent Recourse
            x_c = lar_recourse.get_recourse(x_0, beta=0., theta_p=theta_p)
            J_c_opt = J.eval(x_c, *theta_p)
            
            # Learning Augmented Recourse
            for beta in betas:
                # Alg 1
                if 'alg1' in params['algs']:
                    x = lar_recourse.get_recourse(x_0, beta=beta, theta_p=theta_p)
                    if params['post_process']:
                        # Post process recourse
                        x = hardmax(x, dataset.cat_features)
                    weights_r, bias_r = lar_recourse.calc_theta_adv(x)
                    theta_r = np.hstack((weights_r, bias_r))
                    
                    J_r = J.eval(x, weights_r, bias_r)
                    J_c = J.eval(x, weights_p, bias_p)
                    robustness = J_r - J_r_opt
                    consistency = J_c - J_c_opt
                    
                    append_result(results_opt, 'OPT', seed, alpha, lamb, i, x_0, theta_0, beta, x, theta_r, p, prediction, J_r[0], J_c[0], robustness[0], consistency[0])
                
                # ROAR
                if 'roar' in params['algs']:
                    x, _ = roar_recourse.get_recourse(x_0, theta_p, beta)
                    if params['post_process']:
                        # Post process recourse
                        x = hardmax(x, dataset.cat_features)
                    weights_r, bias_r = lar_recourse.calc_theta_adv(x)
                    theta_r = np.hstack((weights_r, bias_r))
                    
                    J_r = J.eval(x, weights_r, bias_r)
                    J_c = J.eval(x, weights_p, bias_p)
                    robustness = J_r - J_r_opt
                    consistency = J_c - J_c_opt
                    
                    append_result(results_roar, 'ROAR', seed, alpha, lamb, i, x_0, theta_0, beta, x, theta_r, p, prediction, J_r[0], J_c[0], robustness[0], consistency[0])
                
    # Save results
    df_results = pd.DataFrame()
    if 'alg1' in params['algs']:
        df_opt = pd.DataFrame(results_opt)
        if params['save_history']:
            print(f'[Alg1] Saving history for {dataset.name} run {seed}')
            df_opt.to_pickle(f'../results/rob_con_tradeoff/history/lr_{dataset.name}_alg1_{seed}_actionable.pkl')
        df_opt_agg = df_opt.groupby(['alg', 'p', 'beta'], as_index=False).mean(True)
        if params['save_results']:
            print(f'[Alg1] Saving results for {dataset.name} run {seed}')
            df_opt_agg.to_pickle(f'../results/rob_con_tradeoff/output/lr_{dataset.name}_alg1_{seed}_actionable.pkl')
        df_results = pd.concat((df_results, df_opt_agg))
    
    if 'roar' in params['algs']:
        df_roar = pd.DataFrame(results_roar)
        if params['save_history']:
            print(f'[ROAR] Saving history for {dataset.name} run {seed}')
            df_roar.to_pickle(f'../results/rob_con_tradeoff/history/lr_{dataset.name}_roar_{seed}_actionable.pkl')
        df_roar_agg = df_roar.groupby(['alg', 'p', 'beta'], as_index=False).mean(True)
        if params['save_results']:
            print(f'[ROAR] Saving results for {dataset.name} run {seed}')
            df_roar_agg.to_pickle(f'../results/rob_con_tradeoff/output/lr_{dataset.name}_roar_{seed}_actionable.pkl')
        df_results = pd.concat((df_results, df_roar_agg))
    
    return df_results
        

In [5]:
def run_experiment(dataset: Dataset, params: dict, results: List):
    alpha = params['alpha']
    
    for seed in params['seeds']:
        (train_data, test_data) = dataset.get_data(seed)
        X_train, y_train = train_data
        X_test, y_test = test_data
        
        base_model = LR()
        base_model.train(X_train.values, y_train.values)
        
        weights_0 = base_model.model.coef_[0]
        bias_0 = base_model.model.intercept_
        theta_0 = np.hstack((weights_0, bias_0))
        
        if seed == 0:
            predictions = generate_lr_predictions(dataset, theta_0, alpha)
        
        recourse_needed_X_train = recourse_needed(base_model.predict, X_train.values)
        recourse_needed_X_test = recourse_needed(base_model.predict, X_test.values)
        
        lar_recourse = LARRecourse(weights=weights_0, bias=bias_0, imm_features=dataset.imm_features, alpha=alpha)
        roar_recourse = ROAR(weights=weights_0, bias=bias_0, alpha=alpha)
        
        params['lamb'] = lar_recourse.choose_lambda(recourse_needed_X_train, base_model.predict, X_train.values)
        lar_recourse.lamb = params['lamb']
        roar_recourse.lamb = params['lamb']
        
        df_results = recourse_runner(seed, recourse_needed_X_test, lar_recourse, roar_recourse, params, dataset, predictions)
        results.append(df_results)

In [7]:
torch.manual_seed(0)

d_results = {}
params = {}
params['alpha'] = 0.5
params['lamb'] = None
params['seeds'] = range(5)
params['algs'] = ['alg1'] # 'alg1', 'roar
params['post_process'] = True
params['save_results'] = True
params['save_history'] = False
params['save_final_results'] = False


datasets = [GermanDataset()]
for dataset in datasets:
    results = []
    
    print(f'Running {dataset.name} data...')
    run_experiment(dataset, params, results)
    
    d_results[dataset.name] = pd.concat(results)
    if params['save_final_results']:
        d_results[dataset.name].to_pickle(f'../results/rob_con_tradeoff/output/lr_{dataset.name}_actionable')
    
    
    print(f'Finished {dataset.name}\n')

Running german data...
Choosing lambda


lambda=0.8: 100%|██████████| 63/63 [00:00<00:00, 45978.97it/s]
Evaluating recourse | alpha=0.5; lambda=0.7: 100%|██████████| 16/16 [00:02<00:00,  5.80it/s]


[Alg1] Saving results for german run 0
Choosing lambda


lambda=0.8: 100%|██████████| 33/33 [00:00<00:00, 51492.57it/s]
Evaluating recourse | alpha=0.5; lambda=0.7: 100%|██████████| 12/12 [00:01<00:00,  6.55it/s]


[Alg1] Saving results for german run 1
Choosing lambda


...

[Alg1] Saving results for german run 2
Choosing lambda


lambda=0.8: 100%|██████████| 61/61 [00:00<00:00, 46962.65it/s]
Evaluating recourse | alpha=0.5; lambda=0.7: 100%|██████████| 15/15 [00:02<00:00,  5.49it/s]


[Alg1] Saving results for german run 3
Choosing lambda


lambda=0.8: 100%|██████████| 57/57 [00:00<00:00, 46914.31it/s]
Evaluating recourse | alpha=0.5; lambda=0.7: 100%|██████████| 14/14 [00:02<00:00,  5.67it/s]

[Alg1] Saving results for german run 4
Finished german



In [13]:
df_results = d_results['german']
df_agg = df_results.groupby(['alg', 'p', 'beta'], as_index=False).mean()
df_im = df_agg.copy()

In [16]:
colors = px.colors.qualitative.Plotly
nc = len(colors)
font_family = 'Times New Roman'
font_color = 'black'
width, height = 720, 540

params = {
    'ROAR': {
        'name': 'ROAR',
        'symbol': 'star',
        'size': 5
    },
    'OPT': {
        'name': 'Alg1',
        'symbol': 'circle',
        'size': 3
    }
}

x_val = 'consistency'
y_val = 'robustness'

fig = go.Figure()
for alg in df_im['alg'].unique():
    for p in df_im['p'].unique():
        df_alg = df_im[(df_im['p'] == p) & (df_im['alg'] == alg)]
        mask = pareto_frontier(df_alg[y_val], df_alg[x_val])
        df_alg = df_alg.iloc[mask]
        
        t = '\\theta_{p}'.format(p=f'{int(p)}')
        a = '{alg}'.format(alg=params[alg]['name'])
        name = r'$\hat{theta} ({a})$'.format(theta=t, a=params[alg]['name'])

        fig.add_trace(go.Scatter(
            x = df_alg[x_val],
            y = df_alg[y_val],
            marker = dict(color=colors[p], symbol=params[alg]['symbol'], size=params[alg]['size']),
            mode = 'markers+lines',
            name = name,
            customdata=df_alg['beta'],
            hovertemplate='consistency: %{x}<br>robustness: %{y}<br>beta: %{customdata}'
        ))

fig.update_xaxes(
    title=dict(
        text=x_val.capitalize(),
        font=dict(
            family=font_family,
            color=font_color,
            size=25
        )
        ), 
    showline=True, 
    mirror=True,
    linecolor='black', 
    gridcolor='lightgrey', 
    zerolinewidth=1,
    zerolinecolor='lightgrey',
    )

fig.update_yaxes(
    title=dict(
        text=y_val.capitalize(),
        font=dict(
            family=font_family,
            color=font_color,
            size=25
        ), 
        ), 
    showline=True, 
    mirror=True,
    linecolor='black', 
    gridcolor='lightgrey',
    zerolinewidth=1,
    zerolinecolor='lightgrey',
    )

fig.update_layout(
    legend=dict(
        # y=0.975, 
        # y=0.0255, 
        x=0.975, 
        y=0.975, 
        xanchor='right',
        font=dict(
            family=font_family,
            color=font_color,
            size=15
            ), 
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='lightgrey',
        borderwidth=1,
        entrywidth=0.1,
        entrywidthmode='pixels',
        ),
    margin=dict(t=5, b=0, l=1, r=5),
    width=width,
    height=height,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(
        tickfont=dict(
            family=font_family,
            color=font_color,
            size=20,
        ),
    ),
    yaxis=dict(
        tickfont=dict(
            family=font_family,
            color=font_color,
            size=20
        )
    )
    )

print('LR', dataset.name.upper(), 'Actionable', 'Hardmax')
fig.show()

LR GERMAN Actionable Hardmax
